# Advanced Interface Tutorial

In [1]:
import pandas as pd
import numpy as np

In [2]:
from gettsim import set_up_policy_environment
from gettsim import compute_taxes_and_transfers
from gettsim import plot_dag

In [3]:
# Load policy parameters and functions
params_dict, policy_func_dict = set_up_policy_environment("05/09/2020")

In this part we will provide a simulated data set and apply GETTSIM to it. We will also see how it is possible to apply GETTSIM to real data sets. In the part before we had a look on a dataset with one individual. Now we will go further and make our data set more detailed by adding data step by step.

In general, GETTSIM requires a Pandas data frame as data input. There are 41 possible input variables (a list of them  [here](https://gettsim.readthedocs.io/en/latest/crosswalk.html)), but which of these you will need depends on what you are willing to compute. So, if you have a real data set, you will most likely have to adjust it, such that:

1. it contains the variables needed to compute your targets
2. these variables are named according to the standard of GETTSIM

If you are not too familiar with the german tax and transfer system, it might be difficult to know which data you have to pass to GETTSIM to let in compute your reults of interest. To figure out which variables you need [this](gettsim.readthedocs.io/en/latest/tutorials/visualizing_the_system.html) guide might be helpful. Below, we will see a small application to the child benefit.

But for now, let us first have a look on some simulated data. We will look on different household structures and then see in an application of GETTSIM how this is important for the child benefit and the Kinderfreibetrag.

#### Simulating some data

First of all, we are creating a base set: 131 individuals with an working income **per month** (as you remember, this is  is marked by the suffix `_m` after `bruttolohn`) reaching from 1500 to 8000.

In [4]:
min_income = 1500
max_income = 8000
steps = 131

df_base = pd.DataFrame(index=pd.RangeIndex(steps))

# Haushalts-ID und Steuersubjekt-ID: In unserem Beispiel kein Unterschied
df_base["hh_id"] = df_base.index
df_base["tu_id"] = df_base.index
# Bruttolohn ist am einfachsten, jetzt zu erstellen und für Kinder auf 0 zu setzen.
df_base["bruttolohn_m"] = np.linspace(min_income, max_income, steps)
# Vermögen, private Krankenversicherung sind dasselbe für alle Haushaltsmitglieder
df_base["vermögen_hh"] = 0
df_base["prv_krankenv"] = False

Our goal is to have in the end three types of households: Pairs with 0, 1 or 2 children. To get there we first append our base data frame four times to itself.

In [5]:
df_2e_2k = df_base.append(df_base).append(df_base).append(df_base).sort_index()
df_2e_2k.index = pd.RangeIndex(len(df_2e_2k), name="p_id")
df_2e_2k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv
p_id,,,,,
0,0,0,1500.0,0,False
1,0,0,1500.0,0,False
2,0,0,1500.0,0,False
3,0,0,1500.0,0,False
4,1,1,1550.0,0,False
...,...,...,...,...,...
519,129,129,7950.0,0,False
520,130,130,8000.0,0,False
521,130,130,8000.0,0,False


We could see this as 131 houdeholds or transfer units with 4 members. But first, 2 of them should be children without income, and there are some further variables relavant for GETTSIM to deal with Kindergeld and Kinderfreibetrag.

To figure out which variables are relevant for e.g. the child benefit, you may have a look at a dagplot. We imported `plot_dag` at the beginning, what makes it possible to create these dagplots. If you are interested in the variables relevant for e.g. `kindergeld_m`, you can have a look at the *ancestors* of `kindergeld_m`:

In [6]:
selector = {"type": "ancestors", "node": "kindergeld_m", "order":15}

plot_dag(functions=policy_func_dict, selectors=selector);

Loading BokehJS ...

In the plot you see not only the relevant variables which need to be given by the data, but also all the functions GETTSIM uses to compute `kindergeld_m`. `"node"`specifies the node of interest, while `"order"` specifies the number of ancestor nodes you want to see. If you want to see all relevant variables, you should set this number to a high level.

You can also plot the descendents of a vaiable or a function by changing `"ancestors"`to `"descendents"`. It is explained more detailed in the tutorial linked at the beginning of this tutorial.

In [7]:
# Adults
for modulus in 0, 1:
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alter"] = 50
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "hat_kinder"] = True
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "kind"] = False

    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "selbstständig"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "eink_selbst_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "ges_rente_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "vermiet_eink_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "kapital_eink_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "prv_rente_beitr_m"] = 0

    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "wohnort_ost"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "jahr_renteneintr"] = 2100
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "betreuungskost_m"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "in_ausbildung"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "behinderungsgrad"] = 0
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "arbeitsstunden_w"] = 40
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alleinerziehend"] = False

# Children
for modulus in 2, 3:
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "bruttolohn_m"] = np.nan
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "kind"] = True
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "hat_kinder"] = False
    df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alleinerziehend"] = False
    

# First child: 10 years old
modulus = 2
df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alter"] = 10

# Second child: 5 years old
modulus = 3
df_2e_2k.loc[df_2e_2k.index % 4 == modulus, "alter"] = 5

# Some variables must be booleans
for col in "kind", "hat_kinder", "alleinerziehend":
    df_2e_2k[col] = df_2e_2k[col].astype('bool')
df_2e_2k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv,alter,hat_kinder,kind,selbstständig,eink_selbst_m,...,vermiet_eink_m,kapital_eink_m,prv_rente_beitr_m,wohnort_ost,jahr_renteneintr,betreuungskost_m,in_ausbildung,behinderungsgrad,arbeitsstunden_w,alleinerziehend
p_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1500.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
1,0,0,1500.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
2,0,0,NaN,0,False,10.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
3,0,0,NaN,0,False,5.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1,1,1550.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
519,129,129,NaN,0,False,5.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
520,130,130,8000.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
521,130,130,8000.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False


In [8]:
# Delete the five year old child to have households with only one child
df_2e_1k = df_2e_2k.query("alter > 5").copy()
df_2e_1k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv,alter,hat_kinder,kind,selbstständig,eink_selbst_m,...,vermiet_eink_m,kapital_eink_m,prv_rente_beitr_m,wohnort_ost,jahr_renteneintr,betreuungskost_m,in_ausbildung,behinderungsgrad,arbeitsstunden_w,alleinerziehend
p_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1500.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
1,0,0,1500.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
2,0,0,NaN,0,False,10.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
4,1,1,1550.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
5,1,1,1550.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,129,129,7950.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
518,129,129,NaN,0,False,10.0,False,True,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
520,130,130,8000.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False


In [9]:
# Delete both children to have households without children
df_2e_0k = df_2e_2k.query("alter > 10").copy()
df_2e_0k

,hh_id,tu_id,bruttolohn_m,vermögen_hh,prv_krankenv,alter,hat_kinder,kind,selbstständig,eink_selbst_m,...,vermiet_eink_m,kapital_eink_m,prv_rente_beitr_m,wohnort_ost,jahr_renteneintr,betreuungskost_m,in_ausbildung,behinderungsgrad,arbeitsstunden_w,alleinerziehend
p_id,,,,,,,,,,,,,,,,,,,,,
0,0,0,1500.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
1,0,0,1500.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
4,1,1,1550.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
5,1,1,1550.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
8,2,2,1600.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
513,128,128,7900.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
516,129,129,7950.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False
517,129,129,7950.0,0,False,50.0,True,False,False,0.0,...,0.0,0.0,0.0,False,2100.0,0.0,False,0.0,40.0,False


Now we have three data frames: One with households with two children, one with households with one child and one with households withour children. We could apply GETTSIM to each of them, if we wanted to do so. But we also can group them in a dict:

In [10]:
data_nr_children = {
    0: df_2e_0k,
    1: df_2e_1k,
    2: df_2e_2k,
}

For the sake of simplicity we will focus on the dataframe with two adults and two children in the further tutorials. Therefore, we save it in CSV formate:

In [11]:
df_2e_2k.to_csv('2p_2c.csv')

We can already apply GETTSIM to the data frames. Our targets are the ID of the transfer unit, the child benefit, the income tax and the Solidaritätszuschlag, each on transfer unit level:

In [19]:
results_nr_children = {}
for nr_children, data in data_nr_children.items():
    results_nr_children[nr_children] = compute_taxes_and_transfers(
        data,
        functions=policy_func_dict,
        params=params_dict,
        #columns_overriding_functions="tu_id",
        targets=[
                #"tu_id",
                "kindergeld_m_tu",
                "eink_st_tu",
                "soli_st_tu",
            ],
    )

results_nr_children[1]

,eink_st_tu,kindergeld_m_tu,soli_st_tu
p_id,,,
0,1611.323123,204,0.000000
1,1611.323123,204,0.000000
2,1611.323123,204,0.000000
4,1843.015567,204,0.000000
5,1843.015567,204,0.000000
...,...,...,...
517,48631.495800,0,2674.732369
518,48631.495800,0,2674.732369
520,49135.495800,0,2702.452369


In [21]:
total_income_tu = df_2e_2k.groupby("tu_id")["bruttolohn_m"].sum()
total_income_tu

tu_id
0       3000.0
1       3100.0
2       3200.0
3       3300.0
4       3400.0
        ...   
126    15600.0
127    15700.0
128    15800.0
129    15900.0
130    16000.0
Name: bruttolohn_m, Length: 131, dtype: float64

In [29]:
results_nr_children[0]["tu_id"]=df_2e_0k["tu_id"]
results_nr_children[1]["tu_id"]=df_2e_1k["tu_id"]
results_nr_children[2]["tu_id"]=df_2e_2k["tu_id"]

results_nr_children[0].drop_duplicates("tu_id").set_index("tu_id")
results_nr_children[1].drop_duplicates("tu_id").set_index("tu_id")
results_nr_children[2].drop_duplicates("tu_id").set_index("tu_id")

,eink_st_tu,kindergeld_m_tu,soli_st_tu
tu_id,,,
0,1611.323123,408,0.000000
1,1843.015567,408,0.000000
2,2108.837502,408,0.000000
3,2377.984073,408,0.000000
4,2649.746369,408,0.000000
...,...,...,...
126,43838.455800,0,2411.115169
127,44342.455800,0,2438.835169
128,44846.455800,0,2466.555169


This is the output GETTSIM computes for the households with two children (as marked by `_tu`, every value is on transfer unit base). It's not very aesthetic, so you might want to adjust it to your purpose. For example, the child benefit is on monthly, but the taxes on yearly basis. Furthermore, all values are replicated for the members of a transfer unit.

At the moment we also have three data frames for the different household types. For a better comparison, we can put them together.

In [30]:
for nr_children in results_nr_children.keys():
    # child benefit yearly by transfer unit
    results_nr_children[nr_children]["kindergeld_tu"] = results_nr_children[nr_children]["kindergeld_m_tu"] * 12
    # delete monthly values
    results_nr_children[nr_children] = results_nr_children[nr_children].drop("kindergeld_m_tu", axis=1)
    # one line per transfer unit
    results_nr_children[nr_children] = results_nr_children[nr_children].drop_duplicates("tu_id").set_index("tu_id")
    #
    if nr_children == 0:
        add = "no children"
    elif nr_children == 1:
        add = "one child"
    elif nr_children == 2:
        add = "two children"
    else:
        raise NotImplementedError(f"n_kinder = {nr_children}")
    results_nr_children[nr_children] = results_nr_children[nr_children].rename(
        columns={
            "kindergeld_tu": "child benefit, " + add,
            "eink_st_tu": "income tax, " + add,
            "soli_st_tu": "Soli, " + add,
        }
    )
    

In [31]:
total_inc_tu = data_nr_children[0].groupby("tu_id")["bruttolohn_m"].sum() * 12
total_inc_tu.name = "total income tax unit"
total_inc_tu.head()

tu_id
0    36000.0
1    37200.0
2    38400.0
3    39600.0
4    40800.0
Name: total income tax unit, dtype: float64

This is the total income per tax unit. We get there by summing up the incomes of every household member. Note that this only works because the monthly working income of the adults is the only income per household.

We now put the total income per tax unit together with the output of GETTSIM for ervery household type:

In [32]:
comparison = total_inc_tu.to_frame().join(results_nr_children[0]).join(results_nr_children[1]).join(results_nr_children[2]).set_index("total income tax unit").round(2)
comparison

,"income tax, no children","Soli, no children","child benefit, no children","income tax, one child","Soli, one child","child benefit, one child","income tax, two children","Soli, two children","child benefit, two children"
total income tax unit,,,,,,,,,
36000.0,1611.32,0.00,0,1611.32,0.00,2448,1611.32,0.00,4896
37200.0,1843.02,0.00,0,1843.02,0.00,2448,1843.02,0.00,4896
38400.0,2108.84,32.97,0,2108.84,0.00,2448,2108.84,0.00,4896
39600.0,2377.98,86.80,0,2377.98,0.00,2448,2377.98,0.00,4896
40800.0,2649.75,141.15,0,2649.75,0.00,2448,2649.75,0.00,4896
...,...,...,...,...,...,...,...,...,...
187200.0,50400.54,2772.03,0,47119.50,2591.57,0,43838.46,2411.12,0
188400.0,50904.54,2799.75,0,47623.50,2619.29,0,44342.46,2438.84,0
189600.0,51408.54,2827.47,0,48127.50,2647.01,0,44846.46,2466.56,0


## Dealing with real data

It is possible to convert a lot of data formats to a pandas data frame. E.g. if your data has .dta format, you could use `pandas.read_Stata()`.